# Process text

**Purpose**: text pre-processing steps in preparation for NLP tasks.

0.  clean text with regex (Fox News only)
1. remove HTML tags
2. remove punctuation
3. remove stopwords
4. clean whitespace

In [1]:
# matplotlib is logged even though disable_existing_loggers=yes in logging_config.yaml
# https://stackoverflow.com/a/51529172/7016397
# workaround is to manually set the level before creating my logger
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

from usrightmedia.shared.loggers import get_logger
LOGGER = get_logger(filename = '01-text-processing', logger_type='main')

- Pre-processing uses the Bulk API through the Python Elasticsearch client.
- Elasticsearch: ["How Big Is Too Big?"](https://www.elastic.co/guide/en/elasticsearch/guide/current/bulk.html#_how_big_is_too_big)
>The entire bulk request needs to be loaded into memory by the node that receives our request, so the bigger the request, the less memory available for other requests. There is an optimal size of bulk request. Above that size, performance no longer improves and may even drop off. The optimal size, however, is not a fixed number. It depends entirely on your hardware, your document size and complexity, and your indexing and search load.
>Fortunately, it is easy to find this sweet spot: Try indexing typical documents in batches of increasing size. When performance starts to drop off, your batch size is too big. A good place to start is with batches of 1,000 to 5,000 documents or, if your documents are very large, with even smaller batches.
>It is often useful to keep an eye on the physical size of your bulk requests. One thousand 1KB documents is very different from one thousand 1MB documents. A good bulk size to start playing with is around 5-15MB in size.

In [2]:
bulksize = 2000

## Import INCA and check doctypes

In [3]:
from inca import Inca
myinca = Inca()
myinca.database.list_doctypes()

2021-12-11 03:49:10,342 - [WARNING] - INCA - (hype_analysis.py).<module>(21) - $DISPLAY environment variable is not set, trying a different approach. You probably are running INCA on a text console, right?
2021-12-11 03:49:10,890 - [WARNING] - INCA - (var_tsa_analysis.py).<module>(21) - $DISPLAY environment variable is not set, trying a different approach. You probably are running INCA on a text console, right?


{'tweets2': 889739,
 'foxnews': 264621,
 'breitbart': 149241,
 'dailycaller': 121822,
 'oneamericanews': 117287,
 'washingtonexaminer': 71710,
 'newsmax': 71146,
 'gatewaypundit': 39401,
 'infowars': 28453,
 'vdare': 19643,
 'dailystormer': 15823,
 'americanrenaissance': 9838,
 'rushlimbaugh': 9396,
 'seanhannity': 5647}

In [4]:
outlet_doctypes = [
    "americanrenaissance",
    "breitbart",
    "dailycaller",
    "dailystormer",
    "foxnews",
    "gatewaypundit",
    "infowars",
    "newsmax",
    "oneamericanews",
    "rushlimbaugh",
    "seanhannity",
    "vdare",
    "washingtonexaminer",
]

## Cleaning steps

#### 0. regex (Fox News only)

- Rule 1: `"\\n[A-Z0-9 :,\\'!@\$\(\)\-\.\?\:\;\/]+(?:\\n|$)"`

    ```
    Removes substrings which begin with a line break ("\n") and end with a line break or end of string ("$") where the substring contains only capitalized letters or common punctuation:
    - generic promo links
    - links to other news content
    - subheadings within article

    Examples:
    - "\nCLICK HERE TO GET THE FOX NEWS APP\n" (unrelated)
    - "\nTRUMP SAYS HE WILL LEAVE OFFICE IF ELECTORAL COLLEGE VOTES FOR BIDEN\n" (unrelated)
    - "\nCLICK HERE FOR MORE SPORTS COVERAGE ON FOXNEWS.COM\n" (unrelated)
    - "\nSTOCK UP\n" (article subheading)
    - "\nWHAT NEEDS HELP\n" (article subheading)
    
    ```


- Rule 2: `"Get all the latest news on coronavirus and more delivered daily to your inbox\. Sign up here"`

    ```
    Removes generic email signup
    ```  

In [ ]:
try:
    rules_fox = [
        {"regexp": "\\n[A-Z0-9 :,\\'!@\$\(\)\-\.\?\:\;\/]+(?:\\n|$)", "replace_with": ""},
        {
            "regexp": "Get all the latest news on coronavirus and more delivered daily to your inbox\. Sign up here",
            "replace_with": "",
        },
    ]

    # generator
    docs_regexp = myinca.processing.multireplace(
        docs_or_query="foxnews",
        field="article_maintext",
        rules=rules_fox,
        save=True,
        new_key="article_maintext_0",
        action="batch",    
        bulksize=bulksize,
    )
    for doc in docs_regexp:
        # runs process on doc
        pass

except Exception as e:
    LOGGER.warning(e)


 94%|█████████▍| 249001/264621 [1:19:20<05:59, 43.40it/s]

#### 1. remove HTML tags
- run Fox News and non-Fox News outlets separately due to different input key

In [ ]:
try:
    # generator
    docs_rmv_html = myinca.processing.remove_html_tags(
        docs_or_query="foxnews",
        field="article_maintext_0",
        save=True,
        new_key="article_maintext_1",
        action="batch",    
        bulksize=bulksize,
    )
    for doc in docs_rmv_html:
        # runs process on doc
        pass
except Exception as e:
    LOGGER.warning(e)


In [ ]:
remaining_doctypes = [doctype for doctype in outlet_doctypes if doctype not in ["foxnews"]]
remaining_doctypes

In [ ]:
for doctype in remaining_doctypes:
    try:
        docs_rmv_html = myinca.processing.remove_html_tags(
            docs_or_query=doctype,
            field="article_maintext",
            save=True,
            new_key="article_maintext_1",
            action="batch",
            bulksize=bulksize,
        )
        for doc in docs_rmv_html:
            # runs process on doc
            pass
    except Exception as e:
        LOGGER.warning(e)


#### 2. remove punctuation

In [ ]:
for doctype in outlet_doctypes:
    try:
        docs_rmv_punc = myinca.processing.remove_punctuation(
            docs_or_query=doctype,
            field="article_maintext_1",
            save=True,
            new_key="article_maintext_2",
            action="batch",
            bulksize=bulksize,
        )
        for doc in docs_rmv_punc:
            # runs process on doc
            pass
    except Exception as e:
        LOGGER.warning(e)


#### 3. remove stopwords

In [ ]:
for doctype in outlet_doctypes:
    try:
        docs_rmv_stopwords = myinca.processing.remove_stopwords(
            stopwords="english",
            docs_or_query=doctype,
            field="article_maintext_2",
            save=True,
            new_key="article_maintext_3",
            action="batch",
            bulksize=bulksize,
        )
        for doc in docs_rmv_stopwords:
            # runs process on doc
            pass
    except Exception as e:
        LOGGER.warning(e)


#### 4. clean whitespace

In [ ]:
for doctype in outlet_doctypes:
    try:
        docs_clean_whitespace = myinca.processing.clean_whitespace(
            docs_or_query=doctype,
            field="article_maintext_3",
            save=True,
            new_key="article_maintext_4",
            action="batch",
            bulksize=bulksize,
        )
        for doc in docs_clean_whitespace:
            # runs process on doc
            pass
    except Exception as e:
        LOGGER.warning(e)
